In [25]:
username = "vvivatshenko"
password = "TwHf42nA"

In [26]:
import selenium
from selenium import webdriver
from selenium.webdriver.edge.options import Options 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# cookie dump
import pickle

# parse
import requests
from bs4 import BeautifulSoup

In [27]:
def save_cookie(browser):
    pickle.dump(browser.get_cookies(), open("cookies.pkl", "wb"))

def load_cookie(browser):
    cookies = pickle.load(open("cookies.pkl", "rb"))
    for cookie in cookies:
        browser.add_cookie(cookie)

In [30]:
edge_options = Options()
#edge_options.add_argument("user-data-dir=selenium") 

browser = webdriver.Edge(options=edge_options)
print(browser.get_cookies())

[]


In [33]:
print(browser.get_cookies())

[{'domain': 'rpd.donstu.ru', 'httpOnly': True, 'name': '.AspNetCore.Cookies', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'CfDJ8Gs7CqRppCdAnvkfHiiiQPPP_Vr7jEVLDMJNVt7SJOiICS2vfoVH82-49shKRTq-pGTN4eN6WRl6dWJxdBjZoFuMIJfFtRg2Piwfy4lMv1n_h_7LpARJUTxMQnw4nc0pzQ_1fJMNwO0SnC4LIWqbuRbs-d5DGVJkx43fRy519B2dGBywSR1zM4um5QB19NC7M3z9UgwqjXNk5i7_6p0D353qnWIrSh3aotT6wwwMewKtyrdezL50zops1FzTk2iIJvOCd7FtvPYOqbHLUfGa3WOcmoh4N-vN19LNjElAq8r4'}]


In [31]:
browser.get('https://rpd.donstu.ru/')

In [ ]:
if "авторизация" in browser.title.lower():
    browser.find_element(By.NAME,"username").clear().send_keys(username)
    browser.find_element(By.NAME,"password").clear().send_keys(password)

In [ ]:
# st_accept = "text/html"
# st_useragent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15"
# headers = {
#    "Accept": st_accept,
#    "User-Agent": st_useragent
# }
# 
# response = requests.get("https://selectel.ru/blog/courses/", headers)
# src = response.text
# print(src)